In [1]:
from dados_de_mercado import Dados_de_mercado
import pandas as pd
from datetime import datetime, timedelta
import time

In [30]:
energia = ['CMIG4', 'CMIG3', 'TAEE11', 'TAEE3', 'TAEE4', 'CPLE6', 'CPLE3']
saneamento = ['CSMG3', 'SAPR11', 'SAPR3', 'SAPR4', 'SBSP3']
banco = ['BRSR3', 'BRSR5', 'BRSR6', 'ITSA3', 'ITSA4', 'SANB11', 'SANB3', 'SANB4', 'BBAS3', 'BBDC4', 'BBDC3', 'ITUB3', 'ITUB4']
seguro = ['BBSE3', 'PSSA3', 'WIZC3']
combustivel = ['UGPA3', 'ENAT3']
telecom = ['VIVT3', 'TIMS3']

empresas_estudadas = [energia, saneamento, banco, seguro, combustivel, telecom]

In [31]:
empresas_estudadas

[['CMIG4', 'CMIG3', 'TAEE11', 'TAEE3', 'TAEE4', 'CPLE6', 'CPLE3'],
 ['CSMG3', 'SAPR11', 'SAPR3', 'SAPR4', 'SBSP3'],
 ['BRSR3',
  'BRSR5',
  'BRSR6',
  'ITSA3',
  'ITSA4',
  'SANB11',
  'SANB3',
  'SANB4',
  'BBAS3',
  'BBDC4',
  'BBDC3',
  'ITUB3',
  'ITUB4'],
 ['BBSE3', 'PSSA3', 'WIZC3'],
 ['UGPA3', 'ENAT3'],
 ['VIVT3', 'TIMS3']]

In [3]:
setores = {0:'energia',
           1: 'saneamento',
           2: 'banco',
           3: "seguro",
           4: 'combustivel',
           5: 'telecom'}

In [4]:
df_list = []
for i, setor in enumerate(empresas_estudadas):
    df = pd.DataFrame()
    df['empresa'] = setor
    df['setor'] = i
    df_list.append(df)

empresas_estudadas = pd.concat(df_list)

In [5]:
setores_list = []
for k, v in setores.items():
    for setor in empresas_estudadas['setor']:
        if k == setor:
            setores_list.append(v)

empresas_estudadas['setor'] = setores_list

In [6]:
lista_empresas = Dados_de_mercado().codigo_cvm(estatal=True)

In [7]:
codigos_cvm = []
e_estatal = []
for ticker in empresas_estudadas['empresa']:
    codigo_b3 = ticker[0:4]
    dados_empresa = lista_empresas[lista_empresas['codigo_b3']==codigo_b3]
    codigo_cvm = dados_empresa['codigo_cvm'].values
    codigos_cvm.append(codigo_cvm[0])
    e_estatal.append(dados_empresa['estatal'].values[0])

empresas_estudadas['codigo_cvm'] = codigos_cvm
empresas_estudadas['estatal'] = e_estatal

In [9]:
on_list = []
pn_list = []
total_list = []
for empresa in empresas_estudadas['codigo_cvm']:
    time.sleep(2)
    num_acoes = Dados_de_mercado().num_acoes(codigo=empresa)
    on_list.append(num_acoes['shares_on'].values[0])
    pn_list.append(num_acoes['shares_pn'].values[0])
    total_list.append(num_acoes['shares_total'].values[0])

In [10]:
empresas_estudadas['acoes_on'] = on_list
empresas_estudadas['acoes_pn'] = pn_list
empresas_estudadas['acoes_total'] = total_list

In [13]:
equivalencia_codigos={}
for i in range(len(empresas_estudadas)):
    equivalencia_codigos[empresas_estudadas.iloc[i]['codigo_cvm']] = empresas_estudadas.iloc[i]['empresa'][0:4]

In [15]:
df_list = []
for empresa in empresas_estudadas['codigo_cvm']:
    time.sleep(2)
    balanco = Dados_de_mercado().balanco(codigo=empresa)
    balanco = balanco[['assets', 'current_assets', 'liabilities', 'current_liabilities', 'equity', 'reference_date']]
    
    balanco['reference_date'] = pd.to_datetime(balanco['reference_date'])
    balanco['year'] = balanco['reference_date'].dt.year
    balanco = balanco.set_index('reference_date')
    balanco = balanco.resample('Y').last()

    for dado in balanco.columns:
        if dado == 'year':
            pass

        else:
            for i in range(len(balanco)):

                balanco_dic = {'cvm_code': empresa,
                            'data_origin': 'balanco',
                            'data': dado,
                            'year': balanco.iloc[i]['year'],
                            'value': balanco.iloc[i][dado],
                            'ticker': equivalencia_codigos[empresa]
                            }
            
                info = pd.DataFrame(balanco_dic,index=[0])
                df_list.append(info)

In [16]:
for empresa in empresas_estudadas['codigo_cvm']:
    time.sleep(2)
    resultado = Dados_de_mercado().resultado(codigo=empresa)
    resultado = resultado[['net_income', 'ebit', 'period_end', 'period_type']]
    resultado['period_end'] = pd.to_datetime(resultado['period_end'])
    resultado['year'] = resultado['period_end'].dt.year
    resultado = resultado.set_index('period_end')

    if len(resultado.period_type.unique())>1:
        print('ATENÇÃO!!!!!!!!!!!!!!!!!!')
        print(empresa)

    for dado in resultado.columns:
        if dado in ['year', 'period_type']:
            pass

        else:
            for i in range(len(resultado)):
                 
                resultado_dic = {'cvm_code': empresa,
                            'data_origin': 'dre',
                            'data': dado,
                            'year': resultado.iloc[i]['year'],
                            'value': resultado.iloc[i][dado],
                            'ticker': equivalencia_codigos[empresa]
                            }
                
                info = pd.DataFrame(resultado_dic,index=[0])
                df_list.append(info)

In [17]:
for empresa in empresas_estudadas['codigo_cvm']:
    time.sleep(2)
    indicadores_fin = Dados_de_mercado().indicadores_financeiros(codigo=empresa)
    indicadores_fin['period_end'] = pd.to_datetime(indicadores_fin['period_end'])
    indicadores_fin['year'] = indicadores_fin['period_end'].dt.year
    indicadores_fin = indicadores_fin.set_index('period_end')
    indicadores_fin = indicadores_fin[['year', 'gross_debt', 'net_debt']]

    for dado in indicadores_fin.columns:
        if dado in ['_']:
            pass

        else:
            for i in range(len(indicadores_fin)):
                 
                dicionario = {'cvm_code': empresa,
                            'data_origin': 'dre',
                            'data': dado,
                            'year': indicadores_fin.iloc[i]['year'],
                            'value': indicadores_fin.iloc[i][dado],
                            'ticker': equivalencia_codigos[empresa]
                            }
                
                info = pd.DataFrame(dicionario,index=[0])
                df_list.append(info)

In [18]:
infos_empresa = pd.concat(df_list)

C:\Users\eduar\AppData\Local\Temp\ipykernel_20984\1839995745.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  infos_empresa = pd.concat(df_list)


In [19]:
infos_empresa = infos_empresa.drop_duplicates()

In [20]:
infos_empresa

,cvm_code,data_origin,data,year,value,ticker
0,2453,balanco,assets,2009.0,3.029365e+10,CMIG
0,2453,balanco,assets,2010.0,3.355583e+10,CMIG
0,2453,balanco,assets,2011.0,3.700888e+10,CMIG
0,2453,balanco,assets,2012.0,3.256988e+10,CMIG
0,2453,balanco,assets,2013.0,2.981414e+10,CMIG
...,...,...,...,...,...,...
0,24929,dre,net_debt,2019.0,NaN,TIMS
0,24929,dre,net_debt,2020.0,-2.300697e+09,TIMS
0,24929,dre,net_debt,2021.0,NaN,TIMS
0,24929,dre,net_debt,2022.0,2.304770e+08,TIMS


In [21]:
df_div_list = []
for empresa in empresas_estudadas['codigo_cvm']:
    time.sleep(2)
    dividendos = Dados_de_mercado().dividendos(codigo=empresa)
    dividendos = dividendos[['record_date', 'amount', 'type', 'ticker', 'cvm_code']]
    dividendos['record_date'] = pd.to_datetime(dividendos['record_date'])
    dividendos['year'] = dividendos['record_date'].dt.year
    dividendos = dividendos.set_index('record_date')

    for i in range(len(dividendos)):
        dicionario = {'cvm_code': empresa,
                            'data_origin': 'dividendos',
                            'data': dividendos.iloc[i]['type'],
                            'year': dividendos.iloc[i]['year'],
                            'value': dividendos.iloc[i]['amount'],
                            'ticker': dividendos.iloc[i]['ticker']
                            }
                
        info = pd.DataFrame(dicionario,index=[0])
        df_div_list.append(info)

In [22]:
dividendos = pd.concat(df_div_list)

In [23]:
df_cotacao_list = []
for empresa in empresas_estudadas['empresa']:
    time.sleep(2)
    cotacoes_dia = Dados_de_mercado().cotacoes(ticker=empresa)
    cotacoes_dia['date'] = pd.to_datetime(cotacoes_dia['date'])
    cotacoes_dia['year'] = cotacoes_dia['date'].dt.year
    cotacoes_dia = cotacoes_dia.set_index('date')

    max_ano = cotacoes_dia['max'].resample('Y').max()
    min_ano = cotacoes_dia['min'].resample('Y').min()
    close_ano = cotacoes_dia['close'].resample('Y').last()
    year = cotacoes_dia['year'].resample('Y').last()

    cotacoes_ano = close_ano.to_frame()
    cotacoes_ano['max'] = max_ano
    cotacoes_ano['min'] = min_ano
    cotacoes_ano['year'] = year
    cotacoes_ano['ticker'] = empresa

    df_cotacao_list.append(cotacoes_ano)


In [24]:
cotacoes = pd.concat(df_cotacao_list)
cotacoes = cotacoes.reset_index()
cotacoes = cotacoes[['close', 'max', 'min', 'year', 'ticker']]

In [25]:
cotacoes

,close,max,min,year,ticker
0,26.71,31.99,23.90,2010,CMIG4
1,33.27,33.55,25.46,2011,CMIG4
2,22.60,49.50,21.85,2012,CMIG4
3,14.01,26.00,14.00,2013,CMIG4
4,13.14,19.96,12.06,2014,CMIG4
...,...,...,...,...,...
413,14.65,15.14,11.42,2020,TIMS3
414,13.15,14.85,11.15,2021,TIMS3
415,12.40,14.63,11.65,2022,TIMS3
416,17.93,18.58,10.75,2023,TIMS3


In [26]:
rodar_codigo_para_criar_equivalencia = False

if rodar_codigo_para_criar_equivalencia:
    equivalencia_unit = []
    for ticker in empresas_estudadas.empresa:
        equivalencia = input(ticker)
        equivalencia = int(equivalencia)
        equivalencia_unit.append(equivalencia)

else:

    equivalencia_unit = [1, 1, 3, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [27]:
empresas_estudadas['equivalencia_unit'] = equivalencia_unit

In [28]:
empresas_estudadas.to_csv('empresas.csv')
infos_empresa.to_csv('infos_empresas.csv')
cotacoes.to_csv('cotacoes.csv')
dividendos.to_csv('dividendos.csv')